In [1]:
import pandas as pd
import numpy as np
from data_acquisition_and_processing import *
from training_models import *

In [2]:
faults_filepath = '../data/J1939Faults.csv'
diagnostics_filepath = '../data/VehicleDiagnosticOnboardData.csv'

In [3]:
unnessecary_cols = ['IgnStatus', 'RecordID', 'ESS_Id', 'eventDescription', 'actionDescription', 'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake', 'ecuSource', 'active', 'activeTransitionCount', 'faultValue', 'MCTNumber', 'Latitude', 'Longitude', 'LocationTimeStamp']

In [4]:
train_df, test_df = clean_data(faults_filepath, diagnostics_filepath, impute_method='KNeighbors', unnecessary_cols=unnessecary_cols)
train_df.head()

Merging faults and diagnostics data...


c:\Users\irela\Documents\NSS_Projects\big-g-haulin-oats\notebooks\data_acquisition_and_processing.py:9: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv(faults_filepath)


Removing service locations and unnecessary columns...
Number of rows before removing service locations:
1187335
Number of rows after removing service locations:
1051146
Unnecessary columns removed:
['IgnStatus', 'RecordID', 'ESS_Id', 'eventDescription', 'actionDescription', 'ecuSoftwareVersion', 'ecuSerialNumber', 'ecuModel', 'ecuMake', 'ecuSource', 'active', 'activeTransitionCount', 'faultValue', 'MCTNumber', 'Latitude', 'Longitude', 'LocationTimeStamp']
Data after removing service locations and unnecessary columns:
       EventTimeStamp   spn fmi EquipmentID  AcceleratorPedal  \
0 2015-02-21 10:47:13   111  17        1439               0.0   
1 2015-02-21 11:34:34   629  12        1439               NaN   
2 2015-02-21 11:35:31  1807   2        1369               NaN   
3 2015-02-21 11:35:33  1807   2        1369               NaN   
4 2015-02-21 11:39:41  4364  17        1674               NaN   

   BarometricPressure  CruiseControlActive  CruiseControlSetSpeed  \
0               1

,EventTimeStamp,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,...,fmi_23,fmi_29,fmi_3,fmi_31,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9
1001106,2018-07-20 09:31:33,1231,0105406655,-0.026563,-1.839029,0.370184,-0.644781,0.539284,0.318816,0.396278,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001107,2018-07-20 09:42:30,1231,0105406655,0.087764,0.807617,0.052179,0.721326,0.455854,-0.181755,0.792972,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
358800,2016-01-31 07:12:25,629,105301976,0.202092,0.244156,0.611429,-0.375501,-1.494321,-0.382785,-1.240226,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
927313,2018-01-26 18:58:08,37,105338729,0.316419,-2.466096,0.041213,-0.513425,-1.129315,0.257644,-0.956231,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
927314,2018-01-26 18:59:36,37,105338729,0.247822,0.040950,-0.090375,0.543995,0.215993,0.475759,-0.047832,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
prepare_data_for_model(train_df, test_df, target_filename='kneighbors_05_13')

Processing target: HoursUntilNextDerate
Saved HoursUntilNextDerate dataframes for training and testing in data/data_ready_for_models/
Processing target: DerateInNextTwoHours
Saved DerateInNextTwoHours dataframes for training and testing in data/data_ready_for_models/
Processing target: DerateInNextTwentyFourHours
Saved DerateInNextTwentyFourHours dataframes for training and testing in data/data_ready_for_models/


In [7]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

train_data_file_path = '../data/data_ready_for_models/DerateInNextTwentyFourHours_kneighbors_05_13_train.csv'
test_data_file_path = '../data/data_ready_for_models/DerateInNextTwentyFourHours_kneighbors_05_13_test.csv'
model = XGBClassifier()
model_name = 'xgb_classifier_please'

predictions_filepath = f'../data/model_output/DerateInNextTwentyFourHours_kneighbors_05_13_{model_name}.csv'

In [6]:
start_mlflow_run(train_data_file_path, test_data_file_path, predictions_filepath, model)

2025/05/13 17:12:12 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/05/13 17:12:12 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
c:\Users\irela\Documents\NSS_Projects\big-g-haulin-oats\notebooks\training_models.py:49: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(test_data_file_path)


ValueError: Invalid classes inferred from unique values of `y`.  Expected: [    0     1     2 ... 12548 12549 12550], got [    1     2     3 ... 14712 14713 14714]

In [4]:
train_df = pd.read_csv(train_data_file_path)
test_df = pd.read_csv(test_data_file_path)
agg_train_df = add_agg_features_to_cols(train_df)
agg_test_df = add_agg_features_to_cols(test_df)
agg_test_df.head()

C:\Users\irela\AppData\Local\Temp\ipykernel_32376\2536667800.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv(train_data_file_path)


Adding time-based features to BarometricPressure with a rolling window of 24 hours
Adding time-based features to DistanceLtd with a rolling window of 24 hours
Adding time-based features to EngineCoolantTemperature with a rolling window of 24 hours
Adding time-based features to EngineLoad with a rolling window of 24 hours
Adding time-based features to EngineOilPressure with a rolling window of 24 hours
Adding time-based features to EngineOilTemperature with a rolling window of 24 hours
Adding time-based features to EngineRpm with a rolling window of 24 hours
Adding time-based features to FuelLtd with a rolling window of 24 hours
Adding time-based features to FuelRate with a rolling window of 24 hours
Adding time-based features to IntakeManifoldTemperature with a rolling window of 24 hours
Adding time-based features to LampStatus with a rolling window of 24 hours
Adding time-based features to BarometricPressure with a rolling window of 24 hours
Adding time-based features to DistanceLtd w

,EventTimeStamp,DerateInNextTwentyFourHours,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,MeanIntakeManifoldTemperatureLast24hr,MaxIntakeManifoldTemperatureLast24hr,MinIntakeManifoldTemperatureLast24hr,StdIntakeManifoldTemperatureLast24hr,SumIntakeManifoldTemperatureLast24hr,MeanLampStatusLast24hr,MaxLampStatusLast24hr,MinLampStatusLast24hr,StdLampStatusLast24hr,SumLampStatusLast24hr
0,2019-01-11 12:51:18,0,96,301,0.259255,0.240032,-0.726386,0.471749,2.572889,-0.161082,...,-0.888662,-0.888662,-0.888662,NaN,-0.888662,1279.000000,1279.0,1279.0,NaN,1279.0
1,2019-01-11 12:51:18,0,829,301,0.259255,0.240032,-0.726386,0.471749,2.572889,-0.161082,...,-0.888662,-0.888662,-0.888662,0.000000,-1.777323,1279.000000,1279.0,1279.0,0.000000,2558.0
2,2019-01-11 13:11:40,0,96,301,0.133495,-2.048977,0.041213,-0.421475,-0.952026,0.635656,...,-0.364634,0.683421,-0.888662,0.907642,-1.093902,937.666667,1279.0,255.0,591.206676,2813.0
3,2019-01-11 13:11:40,0,829,301,0.133495,-2.048977,0.041213,-0.421475,-0.952026,0.635656,...,-0.102620,0.683421,-0.888662,0.907642,-0.410481,767.000000,1279.0,255.0,591.206676,3068.0
4,2019-01-11 13:13:38,0,96,301,0.259255,0.240074,-0.397415,-0.611942,-0.034297,-0.289759,...,-0.077355,0.683421,-0.888662,0.788069,-0.386773,869.400000,1279.0,255.0,560.867899,4347.0


In [5]:
#agg_test_df['LampStatus'] =  agg_test_df['LampStatus'].astype('category').astype('str')
#agg_train_df['LampStatus'] =  agg_train_df['LampStatus'].astype('category').astype('str')
agg_test_df = agg_test_df.drop(columns='LampStatus')
agg_train_df = agg_train_df.drop(columns='LampStatus')
agg_test_df.head()

,EventTimeStamp,DerateInNextTwentyFourHours,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,MeanIntakeManifoldTemperatureLast24hr,MaxIntakeManifoldTemperatureLast24hr,MinIntakeManifoldTemperatureLast24hr,StdIntakeManifoldTemperatureLast24hr,SumIntakeManifoldTemperatureLast24hr,MeanLampStatusLast24hr,MaxLampStatusLast24hr,MinLampStatusLast24hr,StdLampStatusLast24hr,SumLampStatusLast24hr
0,2019-01-11 12:51:18,0,96,301,0.259255,0.240032,-0.726386,0.471749,2.572889,-0.161082,...,-0.888662,-0.888662,-0.888662,NaN,-0.888662,1279.000000,1279.0,1279.0,NaN,1279.0
1,2019-01-11 12:51:18,0,829,301,0.259255,0.240032,-0.726386,0.471749,2.572889,-0.161082,...,-0.888662,-0.888662,-0.888662,0.000000,-1.777323,1279.000000,1279.0,1279.0,0.000000,2558.0
2,2019-01-11 13:11:40,0,96,301,0.133495,-2.048977,0.041213,-0.421475,-0.952026,0.635656,...,-0.364634,0.683421,-0.888662,0.907642,-1.093902,937.666667,1279.0,255.0,591.206676,2813.0
3,2019-01-11 13:11:40,0,829,301,0.133495,-2.048977,0.041213,-0.421475,-0.952026,0.635656,...,-0.102620,0.683421,-0.888662,0.907642,-0.410481,767.000000,1279.0,255.0,591.206676,3068.0
4,2019-01-11 13:13:38,0,96,301,0.259255,0.240074,-0.397415,-0.611942,-0.034297,-0.289759,...,-0.077355,0.683421,-0.888662,0.788069,-0.386773,869.400000,1279.0,255.0,560.867899,4347.0


In [6]:
new_agg_train_data_file_path = '../data/data_ready_for_models/agg_nolampstatus_DerateInNextTwentyFourHours_kneighbors_05_13_train.csv'
new_agg_test_data_file_path = '../data/data_ready_for_models/agg_nolampstatus_DerateInNextTwentyFourHours_kneighbors_05_13_test.csv'

#agg_test_df.to_csv(new_agg_test_data_file_path)
#agg_train_df.to_csv(new_agg_train_data_file_path)

In [7]:
def start_mlflow_run(train_data_file_path, test_data_file_path, predictions_filepath, model, dropna=True):
    """
    Starts an MLflow run and logs the parameters and artifacts.
    """
    mlflow.set_experiment("big-g-haulin-oats")
    mlflow.autolog()

    with mlflow.start_run():
        mlflow.log_param("train_data_file_path", train_data_file_path)
        mlflow.log_param("test_data_file_path", test_data_file_path)

        mlflow.log_artifact(train_data_file_path, artifact_path="data")
        mlflow.log_artifact(test_data_file_path, artifact_path="data")

        train_df = agg_train_df
        test_df = agg_test_df

        train_df = ensure_target_cols_removed(train_df)
        test_df = ensure_target_cols_removed(test_df)

        if dropna:
            train_df = train_df.dropna()
            test_df = test_df.dropna()
        
        X_train, y_train = split_X_y(train_df)
        X_test, y_test = split_X_y(test_df)

        # Fit the model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Calculate metrics on the test set
        precision, recall, f1, accuracy = evaluate_model(y_test, y_pred)
        print(f"Test precision: {precision}")
        print(f"Test recall: {recall}")
        print(f"Test F1 score: {f1}")
        print(f"Test accuracy: {accuracy}")
        # Log custom metrics for the test set
        mlflow.log_metric("test_precision", precision)
        mlflow.log_metric("test_recall", recall)
        mlflow.log_metric("test_f1", f1)
        mlflow.log_metric("test_accuracy", accuracy)

        print("Model training and evaluation completed successfully.")

        # Save predictions to a CSV file
        predictions_df = pd.DataFrame({
            'Actual': y_test,
            'Predicted': y_pred
        })
        predictions_df.to_csv(predictions_filepath, index=False)
        mlflow.log_artifact(predictions_filepath, artifact_path="data")

        revenue_tp, revenue_fp = count_predictions(predictions_df, agg_test_df)
        total_revenue = revenue_calc(revenue_tp, revenue_fp)

        # Log business metrics
        mlflow.log_metric("true_positives", revenue_tp)
        mlflow.log_metric("false_positives", revenue_fp)
        mlflow.log_metric("revenue", total_revenue)

    return predictions_df

In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV
import mlflow
from xgboost import XGBClassifier

model = XGBClassifier()
model_name = 'xgbclassifier_new_test_3'

predictions_filepath = f'../data/model_output/agg_new_DerateInNextTwentyFourHours_kneighbors_05_13_{model_name}.csv'

predictions_df = start_mlflow_run(new_agg_train_data_file_path, new_agg_test_data_file_path, predictions_filepath, model)


2025/05/13 17:08:09 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/05/13 17:08:09 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
2025/05/13 17:08:27 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\irela\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missi

Test precision: 0.7087378640776699
Test recall: 0.2347266881028939
Test F1 score: 0.3526570048309179
Test accuracy: 0.9970198380926963
Model training and evaluation completed successfully.
True Positives: 2
False Positives: 14
Total revenue: 1000


In [9]:
tp, fp = count_predictions(predictions_df, agg_test_df)
revenue_calc(tp, fp)

True Positives: 2
False Positives: 14
Total revenue: 1000


1000

In [ ]:
agg_train_data_file_path = '../data/data_ready_for_models/agg_DerateInNextTwentyFourHours_kneighbors_05_13_train.csv'
agg_test_data_file_path = '../data/data_ready_for_models/agg_DerateInNextTwentyFourHours_kneighbors_05_13_test.csv'
model = DecisionTreeClassifier()
model_name = 'dct'

#agg_test_df.to_csv(agg_train_data_file_path)
#agg_train_df.to_csv(agg_test_data_file_path)

predictions_filepath = f'../data/model_output/agg_DerateInNextTwentyFourHours_kneighbors_05_13_{model_name}.csv'

In [15]:
predictions_df = start_mlflow_run(agg_train_data_file_path, agg_test_data_file_path, predictions_filepath, model)

2025/05/13 11:58:30 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/05/13 11:58:30 INFO mlflow.tracking.fluent: Autologging successfully enabled for xgboost.
c:\Users\irela\Documents\NSS_Projects\big-g-haulin-oats\notebooks\training_models.py:48: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv(test_data_file_path)
2025/05/13 11:59:20 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\irela\anaconda3\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. A

KeyboardInterrupt: 

In [15]:
final_predictions_df = pd.read_csv('../data/model_output/final_predictions_for_project.csv')
final_predictions_df.head()

,Actual,Predicted
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0


In [16]:
final_test_df = pd.read_csv('../data/data_ready_for_models/DerateInNextTwentyFourHours_kneighbors_05_13_test.csv')
final_test_df.head()

,DerateInNextTwentyFourHours,EventTimeStamp,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,...,fmi_23,fmi_29,fmi_3,fmi_31,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9
0,0,2019-01-08 19:39:29,639,105338729,-0.083727,-1.773036,0.425012,2.245062,-0.347159,0.665821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2019-01-08 19:39:29,639,105338729,-0.083727,-1.773036,0.425012,2.245062,-0.347159,0.665821,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0,2019-01-08 19:42:39,639,105338729,0.042033,-0.492389,0.414046,0.570266,-0.003011,0.244141,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2019-01-08 19:42:39,639,105338729,-0.072294,0.681236,-2.480896,-0.769570,-0.034297,-0.509773,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,2019-01-16 19:41:28,37,105338729,-0.140890,-1.760107,0.260527,-0.546264,-1.077171,0.077285,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
tp_count, fp_count = count_predictions(final_predictions_df, final_test_df)

True Positives: 5
False Positives: 24


In [21]:
revenue_calc(tp_count, fp_count)

Total revenue: 8000


8000